# Calling API to retrieve JSONs for exchange rate on historical dates

In [ ]:
import os
import pandas as pd
from zipfile import ZipFile

In [ ]:
# setting up csv to be able to retrieve information

In [ ]:
# retrieve unique currencies from database

In [ ]:
# retrieve all unique dates in or after 1999 from database

In [ ]:
# setting up API calling parameters
url = 'https://api.exchangeratesapi.io/v1/'
key = ''
base = 'USD'
symbols = ''

# Changing Retrieved Jsons into Dictionary/CSV